In [1]:
!pip install accelerate peft trl datasets bitsandbytes auto-gptq optimum -q

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.get_device_name()
    print(f"CUDA device: {device}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("CUDA is not available on this system.")

CUDA device: Tesla T4
CUDA version: 11.8


In [3]:
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM,GPTQConfig, TrainingArguments
from peft import LoraConfig,prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:
dataset = load_dataset("DR-DRR/Medical_Customer_care",split='train')
dataset['text'][0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/DR-DRR--Medical_Customer_care-e19aadc132a87f29/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


"<s>[INST] Explain the importance of staying hydrated and its benefits for overall health. [/INST] Staying hydrated is crucial for maintaining your overall health. It helps regulate your body temperature, keeps your joints functioning smoothly, aids in digestion, and flushes out toxins. When you're properly hydrated, you'll notice improvements in your skin's appearance and feel more energetic. So, remember to drink water regularly throughout the day to reap these health benefits and keep dehydration symptoms like a dry mouth and dizziness at bay.</s>"

In [5]:
model_ckpt = "TheBloke/Llama-2-7b-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(
    model_ckpt
)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
quantization_config = GPTQConfig(bits=4,disable_exllama=True,tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    revision='main',
    quantization_config=quantization_config,
    device_map='auto')
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [7]:
lora_config = LoraConfig(r=16,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        bias='none',
                        task_type='CAUSAL_LM',
                        target_modules=[
                                    "q_proj",
                                    "k_proj",
                                    "v_proj",
                                    "o_proj",
                                    "gate_proj",
                                    "up_proj",
                                    "down_proj",
                                        ]
)
model = get_peft_model(model,lora_config)

In [8]:
training_args = TrainingArguments(output_dir='.',
                                 dataloader_drop_last=True,
                                 save_strategy='epoch',
                                 num_train_epochs=1,
                                 logging_steps=100,
                                 max_steps=2000,
                                 per_device_train_batch_size=1,
                                 learning_rate=3e-4,
                                 lr_scheduler_type='cosine',
                                 warmup_steps=100,
                                 fp16=True,
                                 #gradient_accumulation_steps=2,
                                 weight_decay=0.05,
                                 report_to=None,
                                 run_name='finetuning-llama2-chat-7b')

In [9]:
trainer = SFTTrainer(model=model,
                    args=training_args,
                    train_dataset = dataset,
                    dataset_text_field='text',
                    max_seq_length=1024,
                    tokenizer=tokenizer,
                    packing=False)

  0%|          | 0/208 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [10]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,2.598800
200,2.412000
300,2.375000
400,2.353100
500,2.396500
600,2.412900
700,2.358500
800,2.321100
900,2.311400
1000,2.338100


TrainOutput(global_step=2000, training_loss=2.3137620620727537, metrics={'train_runtime': 4515.1375, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.443, 'total_flos': 492945782784000.0, 'train_loss': 2.3137620620727537, 'epoch': 0.01})

In [11]:
from huggingface_hub import notebook_login, HfApi

notebook_login()

In [12]:
api = HfApi()

api.upload_folder(
    folder_path='/working/checkpoint-2000',
    path_in_repo=".",
    repo_id="Neupane9Sujal/llama-gptq-medical-finetuned-chatbot",
    repo_type='model',
    create_pr=1
)

rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/627 [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/320M [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.16k [00:00<?, ?B/s]

'https://huggingface.co/Neupane9Sujal/llama-gptq-medical-finetuned-chatbot/tree/refs%2Fpr%2F1/.'

## Inference

In [13]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_ckpt,
    revision='main',
   quantization_config=quantization_config,
    device_map='auto')

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [14]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/working/checkpoint-2000")

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, add_bos_token=True, trust_remote_code=True)

In [16]:
eval_prompt = "[INST] Discuss the role of exercise in maintaining a healthy weight and its impact on overall well-being. [/INST]"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] Discuss the role of exercise in maintaining a healthy weight and its impact on overall well-being. [/INST] Exercise is an important component of any weight loss program, as it helps to burn calories and build muscle mass. Regular physical activity can also help to improve cardiovascular function, reduce stress levels, boost mood, and enhance sleep quality. In addition, regular exercise has been shown to have numerous other benefits for overall health and wellness, including reducing blood pressure, improving insulin sensitivity, and lowering cholesterol levels. While di


## Output



[INST] Discuss the role of exercise in maintaining a healthy weight and its impact on overall well-being. [/INST] Exercise is an important component of any weight loss program, as it helps to burn calories and build muscle mass. Regular physical activity can also help to improve cardiovascular function, reduce stress levels, boost mood, and enhance sleep quality. In addition, regular exercise has been shown to have numerous other benefits for overall health and wellness, including reducing blood pressure, improving insulin sensitivity, and lowering cholesterol levels. While di